In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pandas as pd
df = pd.read_csv('iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
df.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [5]:
df.duplicated().sum()

1

In [6]:
cat = list(df.columns[df.dtypes=='object'])
con = list(df.columns[df.dtypes!='object'])

In [7]:
cat

['species']

In [8]:
con

['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

### seperate X and Y Feature

In [9]:
X = df.drop(columns=['species'])
Y = df[['species']]

In [10]:
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [11]:
Y.head()

,species
0,setosa
1,setosa
2,setosa
3,setosa
4,setosa


### Preprocessing on X feature

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [13]:
num_pipe = Pipeline(steps=[('imputer',SimpleImputer(strategy='mean')),
                           ('scaler',StandardScaler())]).set_output(transform='pandas')

In [14]:
X_pre = num_pipe.fit_transform(X)
X_pre.head()

,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444


### Apply Ordinal Encoder on Y Feature

In [28]:
from sklearn.preprocessing import OrdinalEncoder
ord = OrdinalEncoder().set_output(transform='pandas')
Y_ord = ord.fit_transform(Y)
Y_ord[0:5]

,species
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


### Apply Train and Test Split

In [29]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_pre, Y_ord, test_size=0.2, random_state=42)

In [30]:
xtrain.head()

,sepal_length,sepal_width,petal_length,petal_width
22,-1.506521,1.249201,-1.567576,-1.315444
15,-0.173674,3.090775,-1.283389,-1.052180
65,1.038005,0.098217,0.364896,0.264142
11,-1.264185,0.788808,-1.226552,-1.315444
42,-1.748856,0.328414,-1.397064,-1.315444


In [31]:
ytrain.head()

,species
22,0.0
15,0.0
65,1.0
11,0.0
42,0.0


In [32]:
xtest.head()

,sepal_length,sepal_width,petal_length,petal_width
73,0.310998,-0.592373,0.535409,0.000878
18,-0.173674,1.709595,-1.169714,-1.183812
118,2.249683,-1.052767,1.785832,1.448832
78,0.189830,-0.362176,0.421734,0.395774
76,1.159173,-0.592373,0.592246,0.264142


In [33]:
ytest.head()

,species
73,1.0
18,0.0
118,2.0
78,1.0
76,1.0


In [34]:
xtrain.shape

(120, 4)

In [35]:
xtest.shape

(30, 4)

In [36]:
ytrain.value_counts()

species
1.0        41
0.0        40
2.0        39
Name: count, dtype: int64

In [37]:
ytest.value_counts()

species
2.0        11
0.0        10
1.0         9
Name: count, dtype: int64

### convert target feature to_categorical

In [39]:
from keras.utils import to_categorical
ytrain_cat = to_categorical(ytrain)
ytest_cat = to_categorical(ytest)

In [41]:
ytrain_cat[0:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [42]:
ytest_cat[0:5]

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

### Build Neural Network

In [43]:
xtrain.shape[1]

4

In [45]:
ytrain.value_counts()

species
1.0        41
0.0        40
2.0        39
Name: count, dtype: int64

In [46]:
ytrain_cat.shape

(120, 3)

In [23]:
# Initialize Neural Network
model = Sequential()
# Add first Hidden Layer
model.add(Dense(units=32, activation='relu',input_shape=(xtrain.shape[1],)))
# add second hidden layer
model.add(Dense(units=16, activation='relu'))
# add funal hidden layer
model.add(Dense(units=1, activation='linear'))

### compile the Model

In [24]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

### Train the Neural Network

In [25]:
import tensorflow as tf

In [26]:
hist = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100)

Epoch 1/100




UnimplementedError: Graph execution error:

Detected at node mean_squared_error/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Deep Learning\repository\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Deep Learning\repository\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Deep Learning\repository\venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Deep Learning\repository\venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 608, in run_forever

  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 1936, in _run_once

  File "C:\